In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        n = self.in_channels
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
train_mean,train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train,train_mean,train_std)

In [6]:
x_valid = normalize(x_valid,train_mean,train_std)

In [7]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [12]:
n,*_ = x_train.shape
c = y_train.max() +1
nh = 32
n,c

(50000, tensor(10))

In [13]:
l1 = nn.Conv2d(1,nh,5)
x = x_valid[:100]

In [14]:
x.shape

torch.Size([100, 1, 28, 28])

In [15]:
def stats(x): return x.mean(),x.std()

In [16]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [17]:
stats(l1.weight), stats(l1.bias)

((tensor(0.0038, grad_fn=<MeanBackward1>),
  tensor(0.1158, grad_fn=<StdBackward0>)),
 (tensor(0.0091, grad_fn=<MeanBackward1>),
  tensor(0.1218, grad_fn=<StdBackward0>)))

In [35]:
stats(l1(x))

(tensor(-0.0378, grad_fn=<MeanBackward1>),
 tensor(0.5906, grad_fn=<StdBackward0>))

Seeme like a problem!!! 

let's try with a leaky relu layer (a=1 is equal to no relu)

In [36]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0399, grad_fn=<MeanBackward1>),
 tensor(1.0658, grad_fn=<StdBackward0>))

kaiming init seems to be working nicely 

le'ts now try it with relu

In [38]:
import torch.nn.functional as F

In [39]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [40]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5308, grad_fn=<MeanBackward1>),
 tensor(1.0550, grad_fn=<StdBackward0>))

We see that with kaiming normal initialization (considering a=0 for a non-leaky relu) it works fine! 

In [41]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2060, grad_fn=<MeanBackward1>),
 tensor(0.3667, grad_fn=<StdBackward0>))

however, for the normal uniform initialization from pytorch,it does not work ok, also because it does not consider anything for relus 

#### Let's create our own Kaiming init

In [44]:
nf,ni,kl,kw = l1.weight.shape
rec_fs = kl*kw
print('number of filters out, number of filters in, receptive fieldsize')
nf,ni,rec_fs

number of filters out, number of filters in, receptive fieldsize


(32, 1, 25)

calculate effective fan+in and fan+out for convolutional layers 

In [45]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [46]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [47]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

gain(1) is for no relu: no change

gain(0) isfir normal relu : just sqrt(2)

we see that gain(sqrt(5.)) is nowhere near close to what we want. it's weird

we need to account for the initialization is uniform, not normal.Let's see how big is the initialization std dev: 

In [48]:
torch.zeros(10000).uniform_(-1,1).std(), 1/math.sqrt(3)

(tensor(0.5773), 0.5773502691896258)

In [49]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        n = self.in_channels
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [52]:
torch.nn.modules.conv.init.kaiming_uniform_??

Signature:
torch.nn.modules.conv.init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in "Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification" - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \sqrt{\frac{6}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either 'fan_in' (default) or 'fan_out'. Choosing `fan_in`
            preserves the magnitude of the variance of the weights in 

Note: The reason we are working with uniform distributions here is because pytorch is using it and we are adapting our code to imitate it. Why is pytorch using uniform? Maybe because of legacy. But also because the drawn values are bounded and reduces the infinitesimal chance that one of the weights is proportionally huge (becuase of the long tails of the normal distrib)

In [54]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [57]:
kaiming2(l1.weight, a=0);
stats(f1(x,a=0))

(tensor(0.5164, grad_fn=<MeanBackward1>),
 tensor(0.9208, grad_fn=<StdBackward0>))

we have a std of 1 and mean of 0.5 (becaus eof the ofsetting relu)

If we do it with a=math.sqrt(5.) we should get the same as what pytorch does:

In [59]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2292, grad_fn=<MeanBackward1>),
 tensor(0.4380, grad_fn=<StdBackward0>))

and we do

### What if we use it in a real case scenario?

Let's see how a  real network behave and how it's std is after many layers 

In [60]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [63]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [64]:
y = y_valid[:100].float()

In [65]:
t = m(x)
stats(t)

(tensor(0.0247, grad_fn=<MeanBackward1>),
 tensor(0.0079, grad_fn=<StdBackward0>))

That is likely to be a really big problem 

let's try now backprop

In [66]:
l = mse(t,y)
l.backward()

In [67]:
stats(m[0].weight.grad)

(tensor(-0.0024), tensor(0.0301))

Again, std dev is nowhere near 1

#### Now let's initialize it ourselfs usign kaiming_uniform

In [68]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in "Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification" - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \sqrt{\frac{6}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either 'fan_in' (default) or 'fan_out'. Choosing `fan_in`
            preserves the magnitude of the variance of the weights in the
            forwar

In [72]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [73]:
t = m(x)
stats(t)

(tensor(0.1464, grad_fn=<MeanBackward1>),
 tensor(0.2857, grad_fn=<StdBackward0>))

In [74]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.1544), tensor(0.4586))

It's not 1 but it's much better than 0.008

## Export

In [75]:
!./notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py


## Trying it this other blog post

https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79

In [41]:
x = torch.randn(512)
for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = a @ x
x.mean(), x.std()

(tensor(0.0129), tensor(0.9307))

now with `tanh()` activation 

In [42]:
def tanh(x): return torch.tanh(x)

In [47]:
x = torch.randn(512)
for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = tanh(a@x)
x.mean(), x.std()

(tensor(-0.0024), tensor(0.0648))

##### Xavier

In [52]:
def xavier(m,h): return torch.Tensor(m,h).uniform_(-1,1)*math.sqrt(6./(m+h))

In [53]:
x = torch.randn(512)
for i in range(100):
    a = xavier(512,512)
    x = tanh(a@x)
x.mean(), x.std()

(tensor(-0.0005), tensor(0.0599))

##### for RELU: Kaiming-he

In [54]:
def relu(x): return x.clamp_min(0.)

In [57]:
def kaiming(m,h): return torch.randn(m,h) * math.sqrt(2./m)

In [69]:
x = torch.randn(512)
for i in range(100):
    a = kaiming(512,512)
    x = relu(a@x)
x.mean(), x.std()

(tensor(0.3517), tensor(0.5187))

##### better RELU

In [70]:
def relu(x): return x.clamp_min(0.) - 0.5

In [79]:
x = torch.randn(512)
for i in range(100):
    a = kaiming(512,512)
    x = relu(a@x)
x.mean(), x.std()

(tensor(-0.2749), tensor(0.3420))